In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
url = 'https://rb.gy/ab50wv'
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1366,768)
driver.get(url)

In [18]:
page = driver.find_element(By.CSS_SELECTOR,'[data-tab_type="STANDINGS"]')
page.click()
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[aria-label='Bagian Klasemen']"))
    )
    source_code = driver.find_element(By.CLASS_NAME, 'Jzru1c').get_attribute('innerHTML')
finally:
    driver.quit()

soup = BeautifulSoup(source_code,'html.parser')
tr = soup.find_all('tr')

header = tr[0].find_all('th')
header = [data.text.strip() for data in header]
header = header[2:len(header)-2]
header

['Klub',
 'TPertandingan yang dimainkan',
 'MMenang',
 'SSeri',
 'KKalah',
 'GMGol dicetak',
 'GKGol melawan',
 'SGSelisih gol']

In [19]:
def clean_header(header: list[str]):
    new_header = []
    for data in header:
        if (data[1].islower()):
                new_data = data
        else:
            while (data[1].isupper()):
                data = data[1:]
            new_data = data        
        new_header.append(new_data)
    return new_header

header = clean_header(header)
header

['Klub',
 'Pertandingan yang dimainkan',
 'Menang',
 'Seri',
 'Kalah',
 'Gol dicetak',
 'Gol melawan',
 'Selisih gol']

In [20]:
df = pd.DataFrame(columns=header)

tr = tr[1:]
for data in tr:
    td = data.find_all('td')
    td_text = [td_text.text.strip() for td_text in td]
    df.loc[len(df)] = td_text[2:len(td_text)-3]

# Add new columns
df.insert(0,'Peringkat',df.index +1)
numerical_column = ['Pertandingan yang dimainkan', 'Menang', 'Seri', 'Kalah', 'Gol dicetak', 'Gol melawan', 'Selisih gol']
for i in numerical_column:
    df[i] = df[i].astype('int')
df['Poin'] = (df['Menang'] * 3) + (df['Seri'])
df

,Peringkat,Klub,Pertandingan yang dimainkan,Menang,Seri,Kalah,Gol dicetak,Gol melawan,Selisih gol,Poin
0,1,Liverpool,19,12,6,1,39,16,23,42
1,2,Arsenal,19,12,4,3,36,18,18,40
2,3,Aston Villa,19,12,3,4,40,25,15,39
3,4,Man City,18,11,4,3,43,21,22,37
4,5,Tottenham,19,11,3,5,39,28,11,36
5,6,West Ham,19,10,3,6,33,30,3,33
6,7,Man Utd,19,10,1,8,21,25,-4,31
7,8,Brighton,19,8,6,5,38,33,5,30
8,9,Newcastle,19,9,2,8,37,25,12,29
9,10,Chelsea,19,7,4,8,31,29,2,25
